In [1]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
import tensorflow as tf
from tqdm import tqdm

In [2]:
def crop_image_from_gray(img,tol=7):
  if img.ndim ==2:
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]
  elif img.ndim==3:
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mask = gray_img>tol        
    check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
    if (check_shape == 0):
      return img
    else:
      img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
      img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
      img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
      img = np.stack([img1,img2,img3],axis=-1)
      return img

def circle_crop_v2(img):
  img = crop_image_from_gray(img)

  height, width, depth = img.shape
  largest_side = np.max((height, width))
  img = cv2.resize(img, (largest_side, largest_side))

  height, width, depth = img.shape

  x = int(width / 2)
  y = int(height / 2)
  r = np.amin((x, y))

  circle_img = np.zeros((height, width), np.uint8)
  cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
  img = cv2.bitwise_and(img, img, mask=circle_img)
  img = crop_image_from_gray(img)

  return img

def preprocess_image(image_path, desired_size=224):
  img = cv2.imread(image_path)
  circle_crop = circle_crop_v2(img)
  bgr2rgb = cv2.cvtColor(circle_crop, cv2.COLOR_BGR2RGB)
  img_pil = Image.fromarray(bgr2rgb)
  im = img_pil.resize((desired_size, )*2, resample=Image.LANCZOS)
  return im

In [ ]:
train_df = pd.read_csv('E:/Pawang Code/Diabetic Retinopathy/train.csv')
N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(train_df['id_code'])):
    x_train[i, :, :, :] = preprocess_image('E:/Pawang Code/Diabetic Retinopathy/train_images/{}.png'.format(image_id))

In [1]:
import numpy as np
from PIL import Image
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121

densenet = DenseNet121(
  weights='C:/Users/Bangik/Documents/Python/diabetic-retinopathy-ml/model/DenseNet-BC-121-32-no-top.h5',
  include_top=False,
  input_shape=(224,224,3)
)

def build_model():
  model = Sequential()
  model.add(densenet)
  model.add(layers.GlobalAveragePooling2D())
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(5, activation='sigmoid'))
  
  model.compile(
      loss='binary_crossentropy',
      optimizer=Adam(lr=0.00005),
      metrics=['accuracy']
  )
  
  return model

model = build_model()
model.load_weights('C:/Users/Bangik/Documents/Python/diabetic-retinopathy-ml/model/model.h5')

def get_pad_width(im, new_shape, is_rgb=True):
  pad_diff = new_shape - im.shape[0], new_shape - im.shape[1]
  t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
  l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
  if is_rgb:
      pad_width = ((t,b), (l,r), (0, 0))
  else:
      pad_width = ((t,b), (l,r))
  return pad_width

def preprocess_image(image_path, desired_size=224):
  im = Image.open(f'{image_path}')
  im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
  
  return im

In [6]:
def prediction(image_path):
  tes_image = np.empty((1, 224, 224, 3), dtype=np.uint8)
  tes_image[0, :, :, :] = preprocess_image(image_path)
  predicted = model.predict(tes_image) > 0.5
  predicted = predicted.astype(int).sum(axis=1) - 1
  return predicted

In [9]:
tes = prediction('E:/Pawang Code/Diabetic Retinopathy/dataset_filter/1/03e25101e8e8.png')
print(tes)

C:\Users\Bangik\AppData\Local\Temp\ipykernel_10988\1883825919.py:45: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  im = im.resize((desired_size, )*2, resample=Image.LANCZOS)


1/1 [==============================] - 0s 239ms/step
[1]
